### Nettoyage Dataset

In [7]:
import pandas as pd
import geopandas as gpd
import warnings

warnings.filterwarnings('ignore')

print("🚀 Démarrage du Débuggage de la Fusion...")

# 1. Chargement des CSV
df_vote = pd.read_csv('amiens_elections_clean.csv', sep=';')
df_rev = pd.read_csv('BASE_TD_FILO_IRIS_2021_DISP.csv', sep=';', dtype={'IRIS': str})
df_chom = pd.read_csv('data_DEFM2022_IRIS.csv', sep=';', dtype={'CODGEO': str})
df_log = pd.read_csv('base-ic-logement-2022.CSV', sep=';', dtype={'IRIS': str})

print(f"📊 Votes chargés : {len(df_vote)} lignes")

# 2. Chargement du GeoJSON
print("🌍 Chargement de la carte (650 Mo)...")
gdf_bv = gpd.read_file('contours-france-entiere-latest-v2.geojson')

# Correction du nom de colonne (on utilise codeCommune d'après ton erreur)
gdf_bv_amiens = gdf_bv[gdf_bv['codeCommune'] == '80021'].copy()
print(f"📍 Bureaux de vote Amiens trouvés : {len(gdf_bv_amiens)}")

if len(gdf_bv_amiens) == 0:
    print("❌ ERREUR : Aucun bureau trouvé pour Amiens. Vérifie si le code est '80021' ou 80021")
    # Test alternatif si le premier échoue
    gdf_bv_amiens = gdf_bv[gdf_bv['codeCommune'].astype(str) == '80021'].copy()

# 3. Récupération des IRIS
url_iris = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/georef-france-iris-millesime/exports/geojson?refine=com_code%3A%2280021%22"
gdf_iris = gpd.read_file(url_iris)
print(f"🏘️ Quartiers IRIS Amiens chargés : {len(gdf_iris)}")

# 4. Jointure Spatiale avec harmonisation du CRS
print("🔗 Jointure Spatiale...")
# IMPORTANT : On met les deux cartes dans le même système de coordonnées
gdf_iris = gdf_iris.to_crs(gdf_bv_amiens.crs)

gdf_bv_amiens['centroid'] = gdf_bv_amiens.geometry.centroid
gdf_points = gdf_bv_amiens.copy().set_geometry('centroid')
join_spatial = gpd.sjoin(gdf_points, gdf_iris, how="left", predicate="within")

# On harmonise le code bureau pour le merge
join_spatial['Code_BV_Match'] = pd.to_numeric(join_spatial['codeBureauVote'], errors='coerce')
print(f"🔗 Après jointure spatiale : {len(join_spatial)} lignes")

# 5. MERGE ÉTAPE PAR ÉTAPE (Vérification)
print("\n--- Vérification des Merges ---")

# Merge 1 : Votes + Géographie
df_step1 = pd.merge(df_vote, join_spatial[['Code_BV_Match', 'iris_code']], 
                    left_on='Code_BV', right_on='Code_BV_Match')
print(f"Step 1 (Votes + Géo) : {len(df_step1)} lignes")

if len(df_step1) > 0:
    df_step1 = df_step1.rename(columns={'iris_code': 'IRIS'})
    
    # Merge 2 : + Revenus
    df_step2 = pd.merge(df_step1, df_rev[['IRIS', 'DISP_MED21', 'DISP_TP6021']], on='IRIS', how='left')
    print(f"Step 2 (+ Revenus) : {len(df_step2)} lignes")
    
    # Merge 3 : + Chômage
    df_step3 = pd.merge(df_step2, df_chom[['CODGEO', 'ABCDE']].rename(columns={'CODGEO':'IRIS'}), on='IRIS', how='left')
    print(f"Step 3 (+ Chômage) : {len(df_step3)} lignes")
    
    # Merge 4 : + Logement
    df_final = pd.merge(df_step3, df_log[['IRIS', 'P22_RP_LOCHLMV', 'P22_RP']], on='IRIS', how='left')
    print(f"Step 4 (+ Logement) : {len(df_final)} lignes")

    # Nettoyage et Calculs
    for col in ['DISP_MED21', 'DISP_TP6021']:
        if df_final[col].dtype == object:
            df_final[col] = df_final[col].str.replace(',', '.').astype(float)

    df_final['Pct_HLM'] = (df_final['P22_RP_LOCHLMV'] / df_final['P22_RP']) * 100
    df_final = df_final.rename(columns={'DISP_MED21': 'Revenu_Median', 'DISP_TP6021': 'Taux_Pauvrete', 'ABCDE': 'Nb_Chomeurs'})

    # On ne fait le dropna qu'à la toute fin
    df_ia = df_final[['Code_BV', 'PCT_RN', 'PCT_UG', 'PCT_ENS', 'Revenu_Median', 'Taux_Pauvrete', 'Nb_Chomeurs', 'Pct_HLM']]
    
    # Vérification avant dropna
    print(f"Colonnes avec des vides : \n{df_ia.isna().sum()}")
    
    df_ia_clean = df_ia.dropna()
    print(f"\n✅ Résultat final : {len(df_ia_clean)} lignes prêtes pour l'IA.")
    
    df_ia_clean.to_csv('dataset_final_amiens_IA.csv', index=False, sep=';')
else:
    print("❌ Le merge a échoué dès l'étape 1. Vérifie les numéros de bureaux de vote.")

🚀 Démarrage du Débuggage de la Fusion...
📊 Votes chargés : 73 lignes
🌍 Chargement de la carte (650 Mo)...
📍 Bureaux de vote Amiens trouvés : 71
🏘️ Quartiers IRIS Amiens chargés : 579
🔗 Jointure Spatiale...
🔗 Après jointure spatiale : 695 lignes

--- Vérification des Merges ---
Step 1 (Votes + Géo) : 0 lignes
❌ Le merge a échoué dès l'étape 1. Vérifie les numéros de bureaux de vote.
